# Demo of Alternating Least Square (ALS) RecSys Explanations

#### Imports

In [ ]:
import pathlib
import os, sys
import random
import pandas as pd
from typing import List, Tuple, Dict, Any

import plotly.express as px
from jupyter_dash import JupyterDash
from dash import Dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

root_working_dir = str(pathlib.Path(os.getcwd()).parent.parent.parent)
working_dir = pathlib.Path(os.getcwd()).parent.parent
os.chdir(working_dir)
sys.path.append(root_working_dir)
print(f"Current working directory is: {working_dir}")

from Code.Demos.Models.Src.ALSExplanation import ALSExplainerDemo
from Code.Demos.Models.Src.ModelBasedExplanation import ModelBasedExplainerDemo
from Code.Utils.Src.Enums import ExplanationType
from Code.Utils.Src.Utils import Helpers
import Code.Model.Src.Constants as c
import Code.DataAccessLayer.Src.Constants as c2



#### Run demo based on cached recommendations and recommendation models

In [ ]:
def runDemoUsingCachedModelDatasets():
    """
    Run demo using cached recommendation and explainer models
    """
    als_best_recommender = Helpers.readPickleFromFile(c.ALS_BEST_TRAIN_RECOMMENDER_PATH)
    als_best_explainer = Helpers.readPickleFromFile(c.ALS_BEST_EXPLANATION_MODEL_PATH)
    best_explanations_df = Helpers.prettifyExplanations(als_best_explainer.explanations_df)
    ModelBasedExplainerDemo.reportDemoResults(
            als_best_explainer.explanation_metrics,
            best_explanations_df,
            als_best_recommender.recommendation_metrics,
            als_best_recommender.recommendations_df)
    return als_best_recommender, als_best_explainer, best_explanations_df

als_best_recommender, als_best_explainer, best_explanations_df = runDemoUsingCachedModelDatasets()

#### Sample 10 users and the corresponding explanations of their recommendations

In [ ]:
def createSampleUserIds(n_users: int=10) -> Tuple[int, List[int]]:
    """
    Creates random sample of users
    :param n_users: Number of users
    """
    n_users = 10
    random.seed(100)
    sample_users = random.sample(set(als_best_recommender.metadata.dataset.userId),n_users)
    return n_users, sample_users

n_users, sample_users = createSampleUserIds()
sample_users

In [ ]:
def convertTupleListToDict(row: pd.Series):
    item, contribution = [], []
    for s in row:
        item.append(s[0])
        contribution.append(s[1])
    modified_row = {
        "item": item,
        "contribution": contribution
    }
    return modified_row

sample_explanations_df = best_explanations_df[best_explanations_df.userId.isin(sample_users)]
sample_explanations_df["explanations"] = sample_explanations_df.explanations.apply(convertTupleListToDict)
sample_explanations_df

#### Get the content being recommended i.e. Movies

In [ ]:
content_df = pd.read_csv(c2.MOVIELENS_ITEM_PATH, sep='|', encoding = "ISO-8859-1", skiprows=0, engine='python', header=None)
content_df = content_df.set_index(0)[[1]]
content_df.columns = ['movie']

In [ ]:
content_df.sample(10)

#### Simple visualization UI of explanation

In [ ]:
def runExplanationVisualizer():
    """
    Runs the Explanation visualization
    """
    app = JupyterDash(__name__)
    #app = Dash(__name__)
    app.layout = html.Div([
        html.P("userId:"),
        dcc.Dropdown(
            id='userId',
            value=sample_users[0],
            options=[{'value': x, 'label': x}
                     for x in sample_users],
            clearable=False
        ),
        html.P("Rank:"),
          dcc.Slider(
            id='rank',
            min=1,
            max=10,
            value=1,
            step=1,
            marks={i:'{}'.format(i) for i in range(1,11)}
        ),
        dcc.Graph(id="pie-chart"),
    ])

    @app.callback(
        Output("pie-chart", "figure"),
        [Input("userId", "value"),
         Input("rank", "value")])
    def generateChart(userId, rank):
        train_dataset = als_best_recommender.train_metadata_clone
        explanations_df = sample_explanations_df[(sample_explanations_df['userId'] == userId) & (sample_explanations_df['rank'] == rank)]
        df = pd.DataFrame.from_dict(explanations_df.explanations.iloc[0])
        df['item'] = list(content_df.loc[train_dataset.getOriginalItemId(df.item)].movie)
        title = "Recommended movie: " + content_df.loc[int(explanations_df.itemId)].movie
        fig = px.pie(df, values='contribution', names='item', title=title)
        return fig

    app.css.config.serve_locally = True
    app.scripts.config.serve_locally = True

    # Run app and display result inline in the notebook
    app.run_server(mode='inline', debug=True,port=8099)

runExplanationVisualizer()